# Model Comparison

# best models for each noise level per dataset

## ADL

In [71]:
import pandas as pd 
import numpy as np
pd.set_option("display.precision", 3)

ADL_RESULTS = '/Users/jayanthboddu/Desktop/data_science/upgrad/MSDS/experiments_feb/results/adl_results_till_20230311.csv'
df = pd.read_csv(ADL_RESULTS)
model_name = 'ADL'

In [72]:
df.columns

Index(['timestamp', 'dataset', 'model_noise_type', 'model_noise_level',
       'test_noise_type', 'test_noise_level', 'noisy_mae', 'noisy_psnr',
       'noisy_ssim', 'denosied_mae', 'denoised_psnr', 'denoised_ssim'],
      dtype='object')

In [73]:
df['PSNR_GAIN'] = df['denoised_psnr'] - df['noisy_psnr']
df['SSIM_GAIN'] = df['denoised_ssim'] - df['noisy_ssim']
df['MAE_REDUCTION'] = df['denosied_mae'] - df['noisy_mae']

### All datasets

In [74]:
# Facies mark analysis 
for dataset in ['faciesmark','interpretation', 'stdata12'] : 
    print(f'Dataset : {dataset}')
    condition = df['dataset'] == dataset
    df_f = df[condition]
    df_f.shape
    
    # best performance
    for metric in ['denoised_ssim', 'denoised_psnr', 'denosied_mae'] : 
        x = df_f.groupby(['test_noise_type', 'test_noise_level'])[metric].agg([np.max, np.min,np.mean, np.std, np.median])
        x = x.style.set_table_attributes("style:'display:inline'").set_caption(f'''{model_name} Performance on {dataset} : {metric.upper()} statistics''').format('{:0.2f}').background_gradient()
        
        display(x)
    # metric statistics 
        
    for metric in ['SSIM_GAIN', 'PSNR_GAIN', 'MAE_REDUCTION'] : 
        x = df_f.groupby(['test_noise_type', 'test_noise_level'])[metric].agg([np.max, np.min,np.mean, np.std, np.median])
        x = x.style.set_table_attributes("style:'display:inline'").set_caption(f'''{model_name} Performance on {dataset} : {metric} statistics''').format('{:0.2f}').background_gradient()
        
        display(x)

Dataset : faciesmark


Dataset : interpretation


Dataset : stdata12


In [75]:
df_f.groupby(['test_noise_type', 'test_noise_level'])['denoised_ssim'].agg([np.max])

amax
test_noise_type test_noise_level       
gaussian        0.001             0.694
                0.010             0.719
                0.025             0.678
                0.050             0.558
                0.100             0.530
                0.250             0.401
                0.500             0.386
lpf             0.000             0.917
mixed           0.000             0.587
poisson         0.050             0.780
truly_random    0.000             0.514

In [76]:
# Top 5 model for each noise type and level 
display(df.dataset.unique().tolist())
# dataset = 'faciesmark'
for dataset in df.dataset.unique().tolist() : 
    for nt in df['test_noise_type'].unique().tolist() : 
        for nl in df['test_noise_level'].unique().tolist() : 
            condition1 = df['test_noise_type'] == nt 
            condition2 = df['test_noise_level'] == nl 
            condition3 = df['dataset'] == dataset
            condition4 = df['SSIM_GAIN'] >= 0 
            condition5 = df['PSNR_GAIN'] >= 0 
            # condition6 = df['MAE_REDUCTION'] >= 0 
            # display(df[condition3].shape)
            
            # cond = condition & condition2 & condition3 & condition4 & condition5 & condition6
            cond = condition3 & condition1 & condition2 & condition4
            # print(f'''
            #         ==========================
            #         dataset : {dataset}
            #         noise_type : {nt}
            #         noise_level : {nl}
            #         ==========================
            #         ''')
        
            x = df[cond][['model_noise_type', 'model_noise_level','denoised_ssim', 'SSIM_GAIN','denoised_psnr','PSNR_GAIN']].sort_values(by=['denoised_ssim', 'SSIM_GAIN','denoised_psnr','PSNR_GAIN'], ascending=[False, False, False, False])[:5    ]
            x = x.drop_duplicates(keep='first')
            x = x.style.set_table_attributes("style:'display:inline'").set_caption(f'''Best {model_name} models for {dataset} with degradation : {nt},{nl}''').background_gradient().hide(axis='index')
            display(x)

['faciesmark', 'interpretation', 'stdata12']

model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
truly_random,0.000000,0.981838,0.105570,43.676220,7.694698


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
gaussian,0.010000,0.968157,0.474581,39.960850,13.977861
truly_random,0.000000,0.960806,0.467229,39.316769,13.333780
poisson,0.050000,0.839812,0.346236,33.504456,7.521467
gaussian,0.050000,0.790532,0.296956,30.941109,4.958120
gaussian,0.100000,0.696639,0.203063,27.755007,1.772018


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
truly_random,0.000000,0.942295,0.639642,36.971626,14.964886
gaussian,0.050000,0.871867,0.569213,34.351570,12.344830
gaussian,0.010000,0.864446,0.561793,33.481041,11.474300
gaussian,0.100000,0.759511,0.456858,30.199816,8.193075
mixed,0.000000,0.660501,0.357848,25.590353,3.583612


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
gaussian,0.050000,0.933024,0.747321,35.737343,16.729658
truly_random,0.000000,0.903054,0.717350,32.576405,13.568720
gaussian,0.100000,0.837070,0.651366,32.788715,13.781031
mixed,0.000000,0.673360,0.487656,25.249376,6.241692
gaussian,0.010000,0.592417,0.406714,26.847404,7.839720


...

model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN


model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN


In [77]:
# Best model for each noise type and level 
display(df.dataset.unique().tolist())
display_columns = ['test_noise_type','test_noise_level', 'model_noise_type', 'model_noise_level','denoised_ssim', 'SSIM_GAIN','denoised_psnr','PSNR_GAIN']
df_best_f = pd.DataFrame(columns=display_columns)
df_best_i = pd.DataFrame(columns=display_columns)
df_best_s = pd.DataFrame(columns=display_columns)
# dataset = 'faciesmark'
for i,dataset in enumerate(df.dataset.unique().tolist()) : 
    for nt in df['test_noise_type'].unique().tolist() : 
        for nl in df['test_noise_level'].unique().tolist() : 
            condition1 = df['test_noise_type'] == nt 
            condition2 = df['test_noise_level'] == nl 
            condition3 = df['dataset'] == dataset
            condition4 = df['SSIM_GAIN'] >= 0 
            condition5 = df['PSNR_GAIN'] >= 0 
            # condition6 = df['MAE_REDUCTION'] >= 0 
            # display(df[condition3].shape)
            
            # cond = condition & condition2 & condition3 & condition4 & condition5 & condition6
            cond = condition3 & condition1 & condition2 & condition4 & condition5
            # print(f'''
            #         ==========================
            #         dataset : {dataset}
            #         noise_type : {nt}
            #         noise_level : {nl}
            #         ==========================
            #         ''')
        
            x = df[cond][display_columns].sort_values(by=['denoised_ssim', 'SSIM_GAIN','denoised_psnr','PSNR_GAIN'], ascending=[False, False, False, False]).head(1)
            # print(x.shape[0])
            if x.shape[0] : 
                
                x = x.drop_duplicates(keep='first')
                # x = x.style.set_table_attributes("style:'display:inline'").set_caption(f'''Best DnCNN models for {dataset} with degradation : {nt},{nl}''').background_gradient().hide_index()
                
            else : 
                condition4 = ( nt == 'gaussian') and (nl == 0.001)
                condition5 = ( nt == 'gaussian') and (nl == 0.010)
                condition1 = ( nt == 'mixed') and (nl == 0.000)
                condition2 = (nt == 'lpf') and (nl == 0.000)
                condition3 = (nt == 'poisson') and (nl == 0.050)
                if condition1 or condition2 or condition3 or condition4 or condition5 : 
                    x = pd.DataFrame.from_dict(columns=display_columns, orient='index', data = {'0' :[nt,nl, *([np.nan]*6)]})
                

            if i ==0 : 
                df_best_f = pd.concat([df_best_f, x], ignore_index=True)
            if i ==1 : 
                df_best_i = pd.concat([df_best_i, x], ignore_index=True)
            if i == 2 : 
                df_best_s = pd.concat([df_best_s, x], ignore_index=True)
            
            
display(df_best_f.style.set_table_attributes("style:'display:inline'").set_caption(f'''{model_name} Performance on Faciesmark''').background_gradient().hide(axis="index"))
display(df_best_i.style.set_table_attributes("style:'display:inline'").set_caption(f'''{model_name} Performance on Interpretation''').background_gradient().hide(axis="index"))
display(df_best_s.style.set_table_attributes("style:'display:inline'").set_caption(f'''{model_name} Performance on Stdata12''').background_gradient().hide(axis="index"))

['faciesmark', 'interpretation', 'stdata12']

test_noise_type,test_noise_level,model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
gaussian,0.001000,truly_random,0.000000,0.981838,0.105570,43.676220,7.694698
gaussian,0.010000,gaussian,0.010000,0.968157,0.474581,39.960850,13.977861
gaussian,0.025000,truly_random,0.000000,0.942295,0.639642,36.971626,14.964886
gaussian,0.050000,gaussian,0.050000,0.933024,0.747321,35.737343,16.729658
gaussian,0.100000,gaussian,0.100000,0.916277,0.811904,33.914047,17.858067
gaussian,0.250000,truly_random,0.000000,0.794863,0.748554,29.887257,17.366793
gaussian,0.500000,gaussian,0.500000,0.819164,0.792137,29.246155,18.807676
mixed,0.000000,truly_random,0.000000,0.943585,0.759909,36.762070,21.000333
poisson,0.050000,truly_random,0.000000,0.976047,0.339713,42.218323,13.176949
lpf,0.000000,lpf,0.000000,0.995671,0.002124,46.953819,1.221554


test_noise_type,test_noise_level,model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
gaussian,0.001000,nan,nan,nan,nan,nan,nan
gaussian,0.010000,gaussian,0.010000,0.686862,0.096456,26.117184,2.818928
gaussian,0.025000,gaussian,0.010000,0.652010,0.248116,24.961868,5.642893
gaussian,0.050000,gaussian,0.050000,0.529922,0.265289,23.291994,6.979979
gaussian,0.100000,gaussian,0.050000,0.517010,0.358982,22.791328,9.441957
gaussian,0.250000,gaussian,0.100000,0.397024,0.320618,20.870350,11.062733
gaussian,0.500000,gaussian,0.500000,0.399024,0.350527,20.817570,13.083316
mixed,0.000000,mixed,0.000000,0.570330,0.310363,23.961784,10.994662
poisson,0.050000,poisson,0.050000,0.726144,0.040777,27.277277,0.918789
lpf,0.000000,nan,nan,nan,nan,nan,nan


test_noise_type,test_noise_level,model_noise_type,model_noise_level,denoised_ssim,SSIM_GAIN,denoised_psnr,PSNR_GAIN
gaussian,0.001000,nan,nan,nan,nan,nan,nan
gaussian,0.010000,gaussian,0.010000,0.718894,0.048506,27.159147,1.675152
gaussian,0.025000,gaussian,0.010000,0.678316,0.196393,26.002321,4.497078
gaussian,0.050000,gaussian,0.010000,0.557825,0.223730,23.699808,5.197861
gaussian,0.100000,gaussian,0.050000,0.529676,0.319837,23.511126,7.951392
gaussian,0.250000,gaussian,0.100000,0.401078,0.295113,21.714703,9.657472
gaussian,0.500000,gaussian,0.500000,0.385910,0.317893,21.441618,11.468356
mixed,0.000000,mixed,0.000000,0.587451,0.261234,24.485800,9.740510
poisson,0.050000,poisson,0.050000,0.779700,0.023966,28.947428,0.403234
lpf,0.000000,nan,nan,nan,nan,nan,nan


In [78]:
df_best_f.iloc[]

SyntaxError: invalid syntax (3580315124.py, line 1)